Data wrangling to-do-list:
<li> (1) Qualtrics was set up with three randomized series for the Snowy Picture Task. This means that an answer for a question may be found in 1 of 3 places in the dataset. Step 1 is to organize the dataset so that each question - regardless of in which series it appeared - is assigned to the same variable.
    <li> (2) Create a dummy variable based on the 'submit page' data for either of manipulation pages on Qualtrics. This dummy variable is necessary to distinguish between the two experimental groups (emotional and reasoned).
        <li> (3) return a  numerical variable that represents how many associations an individual case came up with for both tasks.

------------------------------------------------------------------------------------------------------------------------------------------

<li> Import the necessary libraries.

In [1]:
import pandas as pd
import numpy as np

<li> Import our data file. the variable that represents the data file will be called 'df' (= <b>D</b>ata<b>F</b>rame)

In [2]:
df = pd.read_csv(r'/Users/Michiel/Desktop/full_data.csv')

<li> Make sure that our object is indeed a Pandas dataframe.

In [3]:
type(df)

pandas.core.frame.DataFrame

<li> Get a snapshot overview of our dataframe to make sure everything imported correctly.

In [4]:
df

,ResponseId,RecordedDate,Progress,Emo_man_Page Submit,Rea_man_1_Page Submit,SPT_S1_Q1,SPT_S1_Q2,SPT_S1_Q5,SPT_S1_Q8,SPT_S1_Q10,...,AUT_S2_QBrick_12,AUT_S2_QBrick_11,AUT_S2_QBrick_10,AUT_S2_QBrick_9,AUT_S2_QBrick_8,AUT_S2_QBrick_4,AUT_S2_QBrick_5,AUT_S2_QBrick_3,AUT_S2_QBrick_2,AUT_S2_QBrick_1
0,Response ID,Recorded Date,Progress,Timing - Page Submit,Timing - Page Submit,"To what extent did you see a figure, object, o...","To what extent did you see a figure, object, o...","To what extent did you see a figure, object, o...","To what extent did you see a figure, object, o...","To what extent did you see a figure, object, o...",...,Come up with as many uses as you can for a bri...,Come up with as many uses as you can for a bri...,Come up with as many uses as you can for a bri...,Come up with as many uses as you can for a bri...,Come up with as many uses as you can for a bri...,Come up with as many uses as you can for a bri...,Come up with as many uses as you can for a bri...,Come up with as many uses as you can for a bri...,Come up with as many uses as you can for a bri...,Come up with as many uses as you can for a bri...
1,"{""ImportId"":""_recordId""}","{""ImportId"":""recordedDate"",""timeZone"":""Europe/...","{""ImportId"":""progress""}","{""ImportId"":""QID70_PAGE_SUBMIT""}","{""ImportId"":""QID68_PAGE_SUBMIT""}","{""ImportId"":""QID80""}","{""ImportId"":""QID84""}","{""ImportId"":""QID94""}","{""ImportId"":""QID104""}","{""ImportId"":""QID111""}",...,"{""ImportId"":""QID310_12""}","{""ImportId"":""QID310_11""}","{""ImportId"":""QID310_10""}","{""ImportId"":""QID310_9""}","{""ImportId"":""QID310_8""}","{""ImportId"":""QID310_4""}","{""ImportId"":""QID310_5""}","{""ImportId"":""QID310_3""}","{""ImportId"":""QID310_2""}","{""ImportId"":""QID310_1""}"
2,R_22PI0Pj2CyHOohf,2020-04-21 11:35:02,100,NaN,28.8,2,3,2,1,5,...,als e,in plantenbak als vulling,kapot gooien,om hete dingen op te zetten,onderzetter,bbq maken met meerdere stenen,tafeltje,raam in gooien,stapelen,bouwen
3,R_2B5v8VLs5iZUyo1,2020-04-21 12:43:27,100,15.295,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,R_Zl4GLjuRDhOZAv7,2020-04-21 13:20:58,100,31.678,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,R_3MxO5UY4uCBtcPy,2020-05-06 00:00:37,100,NaN,10.998,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
134,R_vJNolB1WjTKrEVr,2020-05-06 11:13:57,100,NaN,21.129,7,6,5,6,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
135,R_3iVkkks0z2a15gP,2020-05-06 11:27:08,100,18.187,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Ter versteviging,Te vervormen tot iets,Bloemen in te steken,Remmend materiaal,Maken van kerststukjes
136,R_2s7w3b88tzDM6j8,2020-05-06 12:16:08,54,12.1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Index 0 and 1 are redundant. They provide equivelant information to our header. The rows represented by these index numbers will be dropped from our dataframe.

In [5]:
df = df.drop(df.index[[0, 1]])

<li> View how many variables are in the data (represented by columns in the data frame)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 136 entries, 2 to 137
Data columns (total 98 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   ResponseId             136 non-null    object
 1   RecordedDate           136 non-null    object
 2   Progress               136 non-null    object
 3   Emo_man_Page Submit    65 non-null     object
 4   Rea_man_1_Page Submit  71 non-null     object
 5   SPT_S1_Q1              47 non-null     object
 6   SPT_S1_Q2              47 non-null     object
 7   SPT_S1_Q5              47 non-null     object
 8   SPT_S1_Q8              47 non-null     object
 9   SPT_S1_Q10             47 non-null     object
 10  SPT_S1_Q12             47 non-null     object
 11  SPT_S2_Q12             44 non-null     object
 12  SPT_S2_Q2              44 non-null     object
 13  SPT_S2_Q10             44 non-null     object
 14  SPT_S2_Q8              44 non-null     object
 15  SPT_S2_Q5              

All of the datatypes in the dataframe are objects. This implies that everything (including the numbers) are represented as string in the dataframe. As a consequence, any mathematical calculations are not possible unless we convert them to integers or floats.

<li> convert objects to floats.

In [7]:
df[df.columns[5:26]] = df[df.columns[5:26]].astype(float)
df[df.columns[2]] = df[df.columns[2]].astype(float)
df[df.columns[27]] = df[df.columns[27]].astype(float)
df[df.columns[31:33]] = df[df.columns[31:33]].astype(float)
df['Duration (in seconds)'] = df['Duration (in seconds)'].astype(float)

<li> Drop all incompleted surveys.
    <li> View how many cases are left afterwards.

In [18]:
df = df.loc[df['Progress'] == 100]
len(df)

125

<li> 'merge' all SPT-task series variables into one.
<li> Turn all NaN (missing values) into a float value 0 in order to be able to succesfully add them.

In [9]:
df['SPT_AG_Q1'] = df.fillna(0)['SPT_S1_Q1'] + df.fillna(0)['SPT_S2_Q1'] + df.fillna(0)['SPT_S3_Q1']

In [10]:
df['SPT_AG_Q2'] = df.fillna(0)['SPT_S1_Q2'] + df.fillna(0)['SPT_S2_Q2'] + df.fillna(0)['SPT_S3_Q2']

In [11]:
df['SPT_AG_Q5'] = df.fillna(0)['SPT_S1_Q5'] + df.fillna(0)['SPT_S2_Q5'] + df.fillna(0)['SPT_S3_Q5']

In [12]:
df['SPT_AG_Q8'] = df.fillna(0)['SPT_S1_Q8'] + df.fillna(0)['SPT_S2_Q8'] + df.fillna(0)['SPT_S3_Q8']

In [13]:
df['SPT_AG_Q10'] = df.fillna(0)['SPT_S1_Q10'] + df.fillna(0)['SPT_S2_Q10'] + df.fillna(0)['SPT_S3_Q10']

In [14]:
df['SPT_AG_Q12'] = df.fillna(0)['SPT_S1_Q12'] + df.fillna(0)['SPT_S2_Q12'] + df.fillna(0)['SPT_S3_Q12']

<li> Create a total score per respondent. We'll need to add all of the columns together.

In [15]:
df['SPT_TOTAL'] = df['SPT_AG_Q1'] + df['SPT_AG_Q2'] + df['SPT_AG_Q5'] + df['SPT_AG_Q8'] + df['SPT_AG_Q10'] + df['SPT_AG_Q12']

The 'Emo_man_Page Submit' variable data to <b>create a dummy variable</b>. 
It can be programmed to display a 0/1 based on the fact that the emotions manipulation page has been displayed to the respondent.
    <li> 1 = emotion manipulation
        <li> 0 = reason manipulation

In [16]:
df['Emo_man_Page Submit'] = df.fillna(1)['Emo_man_Page Submit']

df['Emo_man_Page Submit'] = pd.get_dummies(df['Emo_man_Page Submit'])

df['Emo_man_Page Submit'] = df['Emo_man_Page Submit'].astype(int)

The association task is set up in a way that each seperate answer-slot (15 slots in total per task) represents a value.
This has to be turned into a total, numerical score.

In [17]:
# seperate the AUT-task variables from the data frame.
# simultaneously tell python to turn missing values into '0' and fields with text into '1'
new_data = df[df.columns[38:98]].notnull().astype('int')

# create a new variable / column to represent the sum of all columns per row 
new_data['TOTAL'] = new_data.sum(axis=1)

# concatenate new_data with our original dataframe 'df' 
concat_data = pd.concat([df, new_data['TOTAL']], axis=1)

# write csv file in order to be able to use it in SPSS
concat_data.to_csv('full_data_clean.csv', header=True, index=True)